In [ ]:
import random
import logging
from typing import List, Dict, Any, Tuple, Set
from owlready2 import ThingClass
from owlready2 import get_ontology, Restriction, And, Or, ThingClass, ObjectPropertyClass


#logger = logging.getLogger(__name__)

# ---------------- SmartAxiomParser remains the same ----------------
# (Assume SmartAxiomParser class from previous code is already defined here)

# ---------------- Instance Generation Helpers ----------------

# ---------------- SmartAxiomParser ----------------
class SmartAxiomParser:
    def __init__(self, ontology):
        self.onto = ontology

    def get_class_expressions_with_properties(self, owl_class: ThingClass, debug: bool = False) -> Set[Tuple[Any, Any]]:
        expressions = set()
        for ancestor in owl_class.mro():
            expressions.update(self._parse_expressions(getattr(ancestor, "is_a", []), debug))
            expressions.update(self._parse_expressions(getattr(ancestor, "equivalent_to", []), debug))
        if debug:
            print(f"\nClass expressions with properties for {owl_class.name}:")
            for prop, obj in expressions:
                print(f"Property: {prop}, Object type: {obj}")
        return expressions

    def _parse_expressions(self, expressions: List[Any], debug: bool = False) -> Set[Tuple[Any, Any]]:
        results = set()
        for expr in expressions:
            if isinstance(expr, list):
                results.update(self._parse_expressions(expr, debug))
            elif isinstance(expr, And):
                results.update(self._parse_expressions(expr.Classes, debug))
            elif isinstance(expr, Or):
                for path in expr.Classes:
                    results.update(self._parse_expressions([path], debug))
            elif isinstance(expr, Restriction):
                results.update(self._handle_restriction(expr, debug))
        return results

    def _handle_restriction(self, restriction: Restriction, debug: bool = False) -> Set[Tuple[Any, Any]]:
        prop = restriction.property
        value = restriction.value
        results = set()
        if isinstance(value, Restriction):
            nested_results = self._handle_restriction(value, debug)
            for _, nested_value in nested_results:
                results.add((prop, nested_value))
        elif isinstance(value, list):
            for v in value:
                if isinstance(v, Restriction):
                    nested_results = self._handle_restriction(v, debug)
                    for _, nested_value in nested_results:
                        results.add((prop, nested_value))
                else:
                    results.add((prop, v))
        else:
            results.add((prop, value))
        if debug:
            print(f"Restriction found: Property = {prop}, Object type = {value}")
        return results

def create_individuals_for_class(onto: Any, cls_name: str, count: int) -> List[Tuple[ThingClass, Any]]:
    """
    Create `count` individuals for a given class and its subclasses.
    Returns a list of (class_obj, individual_obj) tuples.
    """
    main_class = onto.search_one(iri=f"*#{cls_name}")
    if main_class is None:
        logger.warning(f"Class '{cls_name}' not found. Skipping.")
        return []
    classes = [main_class]
    for acls in main_class.subclasses():
        if not isinstance(acls, ObjectPropertyClass):
            classes.append(acls)
    print ("classes ", classes)
    individuals = []

    for i in range(count):
        cls_to_instantiate = random.choice(classes)
        new_individual = cls_to_instantiate(f"{cls_to_instantiate.name}_{i}")
        individuals.append((cls_to_instantiate, new_individual))
        logger.info(f"Created individual '{new_individual.name}' of class '{cls_to_instantiate.name}'.")

    return individuals

def get_required_object_classes(parser: Any, subj_cls_obj: ThingClass) -> List[ThingClass]:
    """
    Use SmartAxiomParser to extract required object classes for a given property.
    """
    expr_props = parser.get_class_expressions_with_properties(subj_cls_obj)
    print (expr_props)
    #required_obj_classes = [obj for p, obj in expr_props]
    return expr_props

def assign_relations(onto: Any, parser: Any, instances: Dict[str, List[Tuple[Any, Any]]],
                     relation_config: List[Tuple[str, str, str]]):
    """
    Assign object properties to individuals based on relation_config and class restrictions.
    """
    instance_counter = {}

    for subj_cls_name, prop_name, obj_class_name in relation_config:
        if subj_cls_name not in instances:
            continue
        subjects = instances[subj_cls_name]
        rel = onto.search_one(iri=f"*{prop_name}")
        if rel is None:
            continue

        for subj_cls_obj, subj_individual in subjects:
            required_obj_classes = get_required_object_classes(parser, subj_cls_obj)
            obj_cls_to_use = None
            for p, obj in required_obj_classes:
                if p == rel:
                    print ("obj_cls_to_use", p, obj)
                    if isinstance(obj, Or):
                        continue
                    obj_cls_to_use = obj
                    
            if obj_cls_to_use == None:
                # Fallback to prop_name-based search if no restrictions exist
                obj_cls_to_use = onto.search_one(iri=f"*#{obj_class_name}")
                print ("obj_cls_to_use", obj_cls_to_use)

            print ("obj_cls_to_use", obj_cls_to_use)
            obj_cls_name = obj_cls_to_use.name
            # Reuse existing instances or create new dynamic individual
            if obj_cls_name in instances and instances[obj_cls_name]:
                new_obj_individual = random.choice(instances[obj_cls_name])[1]
            else:
                instance_counter[obj_cls_name] = instance_counter.get(obj_cls_name, 0) + 1
                new_obj_individual = obj_cls_to_use(f"{obj_cls_name}_dynamic_{instance_counter[obj_cls_name]}")
                instances.setdefault(obj_cls_name, []).append((obj_cls_to_use, new_obj_individual))

            rel[subj_individual].append(new_obj_individual)
            logger.info(f"Linked '{subj_individual.name}' -> '{new_obj_individual.name}' via '{prop_name}'.")

# ---------------- Main Instance Generation ----------------

def generate_instances(onto: Any, class_config: Dict[str, int], relation_config: List[Tuple[str, str, str]]) -> Dict[str, List[Tuple[Any, Any]]]:
    """
    Main function to generate individuals and assign relations.
    """
    instances: Dict[str, List[Tuple[Any, Any]]] = {}
    parser = SmartAxiomParser(onto)

    # Step 1: Create individuals
    for cls_name, count in class_config.items():
        individuals = create_individuals_for_class(onto, cls_name, count)
        instances.setdefault(cls_name, []).extend(individuals)

    # Step 2: Assign relations
    assign_relations(onto, parser, instances, relation_config)

    return instances


In [ ]:
# Example: Replace with your class
dataset_name ="OWL2DL-1"
tbox_path: str = f"../../data/tbox/{dataset_name}_TBOX.owl"
onto = get_ontology(tbox_path).load()
my_class = onto.Person  
parser = SmartAxiomParser(onto)

for aclass in my_class.subclasses():
    expr_props = parser.get_class_expressions_with_properties(aclass, debug=True)

In [ ]:
from owlready2 import get_ontology, sync_reasoner_pellet, default_world, Restriction, And, Or # Import all necessary owlready2 functions/classes explicitly
from rdflib import Graph # Used for RDF graph manipulation and counting triples
import logging
from typing import Dict, List, Tuple, Any

# Assuming the user's previously defined helper functions are available here:
# from your_utils_module import generate_instances # Example import if in a separate file

# --- Logging Setup ---
# Set up a basic logger for feedback during script execution
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG) 
# Note: For production use, you would typically configure a handler (like StreamHandler)
# logging.basicConfig(level=logging.INFO) 

# --- Configuration ---
# Define the dataset name. Only one should be uncommented at a time.
# The selected dataset name determines the TBOX file to load.
# dataset_name = "pizza"
#dataset_name = "pizza"
dataset_name = "OWL2DL-1"

logger.info(f"Configuration set for dataset: **{dataset_name}**")

# --- Instance Generation Configuration (Abox) ---

# Define the set of configurations for different experiments.
# The active configuration is selected based on `dataset_name`.

# Configuration for the 'pizza' ontology
pizza_config: Dict[str, Any] = {
    "class_config": {
        "NamedPizza": 100  
    },
    "relation_config": {
        ("NamedPizza", "hasTopping", "PizzaTopping"): 10
}
}

# Configuration for the 'family' ontology (Currently active)
family_config: Dict[str, Any] = {
    "class_config": {
        "Person": 10  
    },
    "relation_config": [
        ("Person", "hasFather", "Person"),
        ("Person", "hasMother", "Person"),
        ("Person", "hasSex", "Sex"),
    ]
}

# Configuration for the 'University' ontology
university_config: Dict[str, Any] = {
    "class_config": {
        "University": 20,
        "Department": 10,
        "Person": 100,
        "Course": 20
    },
    "relation_config": [
        ("University", "hasDepartment", "Department"),
        ("Person", "hasDoctoralDegreeFrom", "University"),
        ("Person", "teachesCourse", "Course"),
        ("Person", "takesCourse", "Course"),
    ]
}

# Select the active configuration based on the dataset_name variable
if dataset_name == "pizza":
    active_config = pizza_config
elif dataset_name == "family":
    active_config = family_config
elif dataset_name == "OWL2DL-1":
    # Assuming OWL2DL-1 uses the university config for this example
    active_config = university_config 
else:
    raise ValueError(f"No configuration defined for dataset '{dataset_name}'.")

class_config: Dict[str, int] = active_config["class_config"]
relation_config: List[Tuple[str, str, str]] = active_config["relation_config"]

# ==========================================================
# 1. Load Ontology (TBox)
# ==========================================================
# Construct the path to the TBox (Terminological Box) file.
tbox_path: str = f"../../data/tbox/{dataset_name}_TBOX.owl"

try:
    # Get the ontology object and load the TBox from the specified path.
    onto = get_ontology(tbox_path).load()
    logger.info(f"Loaded ontology successfully: **{onto.base_iri}** from path: {tbox_path}")
except FileNotFoundError:
    logger.error(f"Error: Ontology file not found at {tbox_path}")
    raise

# ==========================================================
# 2. Generate Individuals (ABox)
# ==========================================================
# Use the previously defined `generate_instances` function to create individuals 
# (ABox assertions) and establish initial relations based on the selected configuration.
# This function populates the ontology object (`onto`) in memory.
logger.info("Generating individuals and initial ABox relations...")
# NOTE: The generate_instances function must be accessible/imported.
try:
    instances: Dict[str, List[Tuple[Any, Any]]] = generate_instances(onto, class_config, relation_config)
    logger.info(f"Finished generating ABox. Total classes instantiated: {len(instances)}")
except NameError:
    logger.error("The `generate_instances` function is not defined or imported.")
    raise

# ==========================================================
# 3. Save and Analyze Ontology BEFORE Reasoning
# ==========================a================================
# Save the ontology state *before* running the reasoner.
temp_file_before: str = f"../../data/abox/{dataset_name}_ABox.owl"
onto.save(temp_file_before, format="rdfxml")
logger.info(f"Ontology state saved to '{temp_file_before}' for pre-reasoning analysis.")